## Connect to Open Library API and fetch book genres 
- for limited amount of books only (books with highest number of rankings)

In [1]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # For progress bar

Import custom function

In [2]:
import sys
sys.path.append('../functions/')

from fun import fetch_genre

### Get list of ISBNs

In [3]:
# Load ISBNs from a CSV file or list
isbn_list = ["9780143126560", "9780062316097", "9781451648539"] 

### Import list of ISBN

In [4]:
df_ratings = pd.read_csv('../data/ISBN_count_rating.csv')
isbn_list = df_ratings['ISBN'].to_list()
isbn_list = list(set(isbn_list)) ## Keep only unique ISBNs

print(f"Number of ISBNs: {len(isbn_list)}")

df_ratings.head(5)

Number of ISBNs: 322158


,ISBN,count_rating
0,971880107,2264
1,316666343,1164
2,385504209,813
3,312195516,668
4,60928336,662


In [14]:
isbn_list_subset = isbn_list[:10000] ##
print(f"Number of selected ISBNs: {len(isbn_list_subset)}")
isbn_list_subset[:5]

Number of selected ISBNs: 10000


['394528093', '3518391143', '812524225', '590317695', '590319574']

#### Function to fetch book genre

In [15]:
fetch_genre("1586601989")

{'ISBN': '1586601989',
 'Title': 'Holy Bible',
 'Genres': "Manuscripts, Latin (Medieval and modern), Glossaries, vocabularies, Early works to 1800, Bible, Religion, Bibles, Biblical Studies - General, Religion / Bible / General, Bible - General, Commentaries, Authorized, Criticism, interpretation, Shorthand, Sacred books, Church of Jesus Christ of Latter-day Saints, BÍBLIA (INTERPRETAÇÃO), Juvenile literature, The Bible, Biblia, Christian life, Study and teaching, English Concordances, Bible, study and teaching, Jews, Antiquities, Apocryphal books (Old Testament), Education, Doctrines, Spiritual life, Religious life, New Testament, Peshitta, English Harmonies, Bible, commentaries, Introductions, Biography, Creation, Biblical teaching, History, Bible, criticism, interpretation, etc., Catholic Truth Society (Great Britain), Church year meditations, Lectionaries, Bible, theology, Bibel, Bijbel, Christianity, Spirituality, Bible. -- Old Testament, Bible. -- New Testament, Teenagers' use, E

In [8]:
# Process ISBNs in parallel
def fetch_all_genres(isbn_list, max_workers=10):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_genre, isbn): isbn for isbn in isbn_list}
        for future in tqdm(as_completed(futures), total=len(isbn_list), desc="Fetching Genres"):
            results.append(future.result())
    return results

In [9]:
# Run the function
#results = fetch_all_genres(isbn_list, max_workers=10)

#### Save results to CSV

In [10]:
#df = pd.DataFrame(results)
#df.to_csv("book_genres.csv", index=False)

#print("✅ Data saved to book_genres.csv")